# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()

load_cells = True
load_meshes = False
cell_radius = 5.0 # Micrometters

### Load Atlas

In [ ]:
population_name = 'atlas_ccfv3a_averaged'

In [ ]:
status = be.reset_scene()
atlas_center = [7075.026185864174,3862.47588645537,5699.969406561653]

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]
cerebellum_granular_layer_ids = [10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732]
cerebellum_molecular_layer_ids =  [10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734]

olfactory_bulb_ids = [507, 212, 228, 236, 244, 220]

medula_ids = [
      773,       781,      1039,       789,      1048,        45,      1069,
      560,       307,        53,       568,        61,       576,       69,
      839,      1098,        76,        77,       83,      1107,       852,
      859,       607,        96,       354, 589508451,       101,       106,
      112,       370,       372,       887,       379,       640,      386,
      642,       903,       135,       136,       651,       395,       653,
      143,       659,       661,       666,       154,       161,       674,
      169,       682,       938,       939,       429,       177,       691,
      437,       185,       955,       445,       701,       193,       963,
      711,       970,       203,       202,       206,       207,       720,
      209,       978,       217,       222,       225,       995,       230,
      235,       765
]

fibers_ids = [960, 1000, 1009, 396, 109, 698]
regions_to_ignore = [
    1557651848, # Cell excluded from region 1557651847 (x<540)
    1024543562, 2358040414, 3263488087, 2416897036,
    3034756217, 2165415682, 2614168502, 1842735199,
    3101970431, 1140764290, 3092369320, 1811993763]
ignore_filter = str(fibers_ids + regions_to_ignore).replace('[','').replace(']','')

In [ ]:
for x in range(2):
    for y in range(2):
        for z in range(2):
            filter = '('
            if x == 0:
                filter += 'x>7075'
            else:
                filter += 'x<=7075'
            filter += ' AND '
            if y == 0:
                filter += 'y>3862'
            else:
                filter += 'y<=3862'
            filter += ' AND '
            if z == 0:
                filter += 'z>5699'
            else:
                filter += 'z<=5699'
            filter += ')'
            atlas_assembly_name = filter
            be.remove_assembly(atlas_assembly_name)
            atlas_assembly = be.add_assembly(atlas_assembly_name)
            atlas_model = be.add_atlas(
                assembly_name=atlas_assembly_name,
                population_name=population_name,
                load_cells=load_cells, load_meshes=load_meshes, cell_radius=cell_radius,
                cell_sql_filter='%s AND region_guid IN (SELECT guid FROM %s.region WHERE guid NOT IN (%s))' % (filter, population_name, ignore_filter),
            )

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % 'atlas_ccfv3a')
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    k = 0
    material_ids = be.get_material_ids(model_id)['ids']
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    refraction_indices = list()
    glossinesses = list()
    for material_id in material_ids:
        c = [1, 0, 0]
        m_id = material_id
        if m_id in region_colors:
            c = region_colors[m_id]
        alpha = [1.0, 1.0, 1.0]

        if material_id in olfactory_bulb_ids:
            value = 0.5 * float(k) / float(len(olfactory_bulb_ids))
            alpha[0] -= value
            alpha[1] -= value
            alpha[2] -= value
            k += 1

        shading_modes.append(be.shading_mode.NONE)
        glossinesses.append(1.0)
        specular_exponents.append(50.0)
        palette.append([alpha[0] * c[0], alpha[1] * c[1], alpha[2] * c[2]])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, 
        opacities=opacities, specular_exponents=specular_exponents,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
background_color = [0.0, 0.0, 0.0]
status = core.set_renderer(
    head_light=True,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
status = core.set_renderer(
    head_light=False,
    current='ambient_occlusion', background_color=[0,0,0],
    subsampling=4, max_accum_frames=128)
params = core.AmbientOcclusionRendererParams()
params.gi_ray_length = 1000
params.gi_samples = 1
params.max_ray_depth = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
core.set_camera(
    current='perspective',
    orientation=[0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
    position=[-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
    target=[6733.589672442965, 4796.273454159725, 6016.635720470601],
)

## Snapshots

In [ ]:
import os
output_folder = '/scratch/images/atlas/%s' % population_name
os.makedirs(output_folder, exist_ok=True)
k = 4
spp = 64

In [ ]:
cameras = [
    [
        [0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
        [-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
        [6733.589672442965, 4796.273454159725, 6016.635720470601],
        'perspective'
    ]
    ,
    [
        [0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
        [-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
        [6733.589672442965, 4796.273454159725, 6016.635720470601],
        'orthographic'
    ]
    ,
    [
        [0.7071067811865474, 0.0, 0.0, 0.7071067811865477],
        [7037.267621994019, -6075.033398596237, 5699.9692382812555],
        [7037.267621994019, 3812.2357177734375, 5699.96923828125],        
        'orthographic'
    ]
]

params = core.OrthographicCameraParams()
params.height = 12000
status = core.set_camera_params(params)

In [ ]:
def show_models(visible_ids):
    model_ids = be.get_model_ids()['ids']
    i = 0
    for model_id in model_ids:
        visible = (visible_ids[i] == 1)
        core.update_model(model_id, visible=visible)
        i += 1

In [ ]:
models_to_show = [
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 1, 1, 0, 0],
]

version = 0
for camera in cameras:
    core.set_camera(
        current=camera[3],
        orientation=camera[0], position=camera[1], target=camera[2]
    )
    for mts in models_to_show:
        show_models(mts)
        mm.create_snapshot(
            renderer='advanced',
            path=output_folder, base_name='%s_radius_%dnm_%s_set1_%03d' % (population_name, cell_radius * 1000, camera[3], version),
            size=[k * 960, k * 540],
            samples_per_pixel=128)
        mm.create_snapshot(
            renderer='ambient_occlusion',
            path=output_folder, base_name='%s_bw_radius_%dnm_%s_set1_%03d' % (population_name, cell_radius * 1000, camera[3], version),
            size=[k * 960, k * 540],
            samples_per_pixel=128)
        version += 1

### Set 2

In [ ]:
cameras = [
    [
        [0.3568657918956826, -0.2889789424949062, 0.878446217895065, 0.132174958426507],
        [18142.5437226011, -5635.84419998349, 16204.894769982748],
        [8028.432262684154, 5423.561490312162, 5582.073515085155],
        'perspective'
    ]
    ,
    [
        [0.3568657918956826, -0.2889789424949062, 0.878446217895065, 0.132174958426507],
        [18142.5437226011, -5635.84419998349, 16204.894769982748],
        [8028.432262684154, 5423.561490312162, 5582.073515085155],
        'orthographic'
    ]
]

params = core.OrthographicCameraParams()
params.height = 12000
status = core.set_camera_params(params)

In [ ]:
models_to_show = [
    [1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1, 0, 1],
    [1, 1, 0, 0, 1, 1, 0, 0],
]

version = 0
for camera in cameras:
    core.set_camera(
        current=camera[3],
        orientation=camera[0], position=camera[1], target=camera[2]
    )
    for mts in models_to_show:
        show_models(mts)
        mm.create_snapshot(
            renderer='advanced',
            path=output_folder, base_name='%s_radius_%dnm_%s_set2_%03d' % (population_name, cell_radius * 1000, camera[3], version),
            size=[k * 960, k * 540],
            samples_per_pixel=128)
        mm.create_snapshot(
            renderer='ambient_occlusion',
            path=output_folder, base_name='%s_bw_radius_%dnm_%s_set2_%03d' % (population_name, cell_radius * 1000, camera[3], version),
            size=[k * 960, k * 540],
            samples_per_pixel=128)
        version += 1